In [6]:
import re

reg="^[0-9]+\.[0-9]+\.[0-9]+\.[0-9]{1,3}$"

ips = ['129.123.1.21',
       'asd.qwe.123.15',
       '123/141.213/412']

for ip in ips:
    if re.match(reg, ip):
        print('True')
    else:
        print('False')

True
False
False


In [27]:
import pyproj
import numpy as np

lat1 = 35 + 50.9048 / 60.0
lon1 = 128 + 29.4713 / 60.0

# WGS84 - Latitude, Longitude
epsg4326 = pyproj.CRS.from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
# UTM-K (GRS80)
epsg5179 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
# TM-WEST
epsg5185 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=125 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
# TM-CENTRAL
epsg5186 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
# TM-EAST
epsg5187 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=129 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")

transformer5179 = pyproj.Transformer.from_crs(epsg4326, epsg5179)
transformer5185 = pyproj.Transformer.from_crs(epsg4326, epsg5185)
transformer5186 = pyproj.Transformer.from_crs(epsg4326, epsg5186)
transformer5187 = pyproj.Transformer.from_crs(epsg4326, epsg5187)


def convert_coordinate(lon, lat, type=5179):
    if type == 5179: # UTM-K
        E, N = transformer5179.transform(lon, lat)
    elif type == 5185:
        E, N = transformer5185.transform(lon, lat)
    elif type == 5186:
        E, N = transformer5186.transform(lon, lat)
    elif type == 5187:
        E, N = transformer5187.transform(lon, lat)
        
    return E, N

print(convert_coordinate(lon1, lat1, type=5179))
print(convert_coordinate(lon1, lat1, type=5185))
print(convert_coordinate(lon1, lat1, type=5186))
print(convert_coordinate(lon1, lat1, type=5187))

(1089505.5119782635, 1761773.9003020208)
(515441.5108954594, 366857.26349295647)
(334712.6938129107, 362251.7414842684)
(154035.83692303533, 361344.4853010551)


In [26]:
import pyproj
import numpy as np
import pandas as pd
import rospkg

rp = rospkg.RosPack()
root_dir = rp.get_path('gnss_ros')

# WGS84 - Latitude, Longitude
epsg4326 = pyproj.CRS.from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
# UTM-K (GRS80)
epsg5179 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
# # TM-WEST
# epsg5185 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=125 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
# # TM-CENTRAL
# epsg5186 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
# # TM-EAST
# epsg5187 = pyproj.CRS.from_string("+proj=tmerc +lat_0=38 +lon_0=129 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")


class GNSSTransformer:
    def __init__(self):
        self.transformer = pyproj.Transformer.from_crs(epsg4326, epsg5179)
        
    def convert_coordinate(self, lon, lat):
        # lon = lon // 100 + lon % 100 / 60.0
        # lat = lat // 100 + lat % 100 / 60.0
        e, n = self.transformer.transform(lon, lat)
        return e, n
    
    def convert(self, df: pd.DataFrame):
        if 'LAT' not in df.columns or 'LON' not in df.columns:
            return False, df
        else:
            df['LAT_DEG'] = df['LAT'] // 1 + (df["LAT"] % 1) * 100 / 60.0
            df["LON_DEG"] = df["LON"] // 1 + (df["LON"] % 1) * 100 / 60.0
            # df['E'], df['N'] = zip(*df[['LON', 'LAT']].apply(self.convert_coordinate))
            df['E'], df['N'] = self.convert_coordinate(df['LON_DEG'], df['LAT_DEG'])
            return True, df
        
transformer = GNSSTransformer()

data = pd.read_csv(root_dir + '/scripts/FITO/CLBY-1B-10804-99_20241108_055906_0.csv')[['LAT', 'LON']]

# print(data.head)

ret, coord = transformer.convert(data)
print(coord.head)

<bound method NDFrame.head of            LAT         LON    LAT_DEG     LON_DEG             E             N
0    35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
1    35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
2    35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
3    35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
4    35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
..         ...         ...        ...         ...           ...           ...
995  35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
996  35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
997  35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
998  35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06
999  35.509048  128.294713  35.848414  128.491189  1.089506e+06  1.761774e+06

[1000 rows x 6 columns]>
